# Computational Practicum: Resource Requirements for Gene Expression in Cell-Free Systems
Fill me in

In [1]:
include("Include.jl")

generate_parameter_dictionary (generic function with 1 method)

## Task 1: Build the System Matricies
In this task, we will build the system matrices for the cell-free system. The system matrices are used to describe the interactions between the different components of the system. 

In [2]:
genesequence, proteinsequence = let

    # setup the paths to the gene and protein sequence files
    path_to_gene_file = joinpath(_PATH_TO_DATA, "deGFP.gene")
    path_to_protein_file = joinpath(_PATH_TO_DATA, "deGFP.prot")

    # load the sequences from the files - 
    gene_sequence = load_gene_sequence_from_file(path_to_gene_file); # load the gene sequence
    protein_sequence = load_protein_sequence_from_file(path_to_protein_file); # load the protein sequence

    # return -
    gene_sequence, protein_sequence
end;

Fill me in


In [3]:
reactions = let

    # initialize -
    reactions = Vector{String}();
    
    # TXTL and "hypothetical" exchange reactions -
    transcription_reactions = transcription(genesequence); # transcription reactions
    translation_reactions = translation(proteinsequence); # translation reactions
    txtl_reactions = [transcription_reactions; translation_reactions]; # combine transcription and translation reactions
    
    # build the exchange reactions -
    exchange_reactions = exchangereactions(txtl_reactions); # exchange reactions
    
    # build the list of reactions -
    reactions = [txtl_reactions; exchange_reactions]; # combine all reactions
end;

Fill me in

In [4]:
let

    # initialize -
    path_to_reaction_file = joinpath(_PATH_TO_DATA, "deGFP.reactions"); # path to the reaction file

    # write the reactions to the file -
    open(path_to_reaction_file, "w") do io
        for reaction in reactions
            println(io, reaction); # write the reaction to the file
        end
    end

end

Fill me in.

In [5]:
S, species, reactions, reaction_dictionary, bounds = let

    # initialize -
    path_to_reaction_file = joinpath(_PATH_TO_DATA, "deGFP.reactions"); # path to the reaction file
    reactions = read_reaction_file(path_to_reaction_file); # read the reactions from the file
    
    # Compute the stoichiometric matrix -
    (S, species_array, reaction_array, reaction_dictionary) = build_stoichiometric_matrix(reactions); # compute the stoichiometric matrix
   
    # compute the bounds -
    bounds = build_default_bounds_array(reactions); # this is the default bounds array

    # return -
    S, species_array, reactions, reaction_dictionary, bounds
end;

Constants

In [6]:
R = 8.314; # universal gas constant (units: J/(mol*K))
T = 37 + 273.15; # temperature (units: K);
β = 1/(R*T); # inverse temperature (units: 1/(J*mol)) - thermodynamic beta in units of 1/(J*mol)
kcat_default = 10.0*(3600); # turnover number (units: 1/hr)
default_enzyme_concentration = 0.01; # enzyme concentration (units: mmol/gDW
parameters = generate_parameter_dictionary(joinpath(_PATH_TO_CONFIGURATION, "Parameters.json")); # load the biophysical parameters (approximately true)

## Task 2: Update the Bounds Array
Fill me in

There are pathologies when using flux balance analysis to model gene expression and signal transduction can cause pathologies. For example, translation occurs without transcription, and transcription occurs without PhoB activation. How can we fix this? 

__Answer__: there is a trick with the bounds (that incorporates many things we have been exploring) that we can use to fix the gene expression problem:
* [Vilkhovoy M, Horvath N, Shih CH, Wayman JA, Calhoun K, Swartz J, Varner JD. Sequence-Specific Modeling of E. coli Cell-Free Protein Synthesis. ACS Synth Biol. 2018 Aug 17;7(8):1844-1857. doi: 10.1021/acssynbio.7b00465. Epub 2018 Jul 16. PMID: 29944340.](https://www.biorxiv.org/content/10.1101/139774v2)

__Fix__: We (equality) bound a transcription rate $\hat{v}_{i}$ as $\hat{r}_{X, i}u_{i} = \hat{v}_{i} = \hat{r}_{X, i}u_{i}$, while a translation rate is bounded from above by the modified kinetic limit: $0\leq\hat{v}_{j}\leq\hat{r}_{L,j}w_{j}$. This is interesting because the kinetic limits are (semi)mechanistic descriptions of the transcription and translation rate. At the same time, the control variables contain continuous Boltzmann-type descriptions of the logical controlling of these processes.

Let's implement this for our PhoR/PhoB model. First, let's compute the kinetic limit of transcription for PhoA:

## Task 3: Estimate the Protein Production Rate
In this task, we will estimate the protein production rate of the cell-free system and the protein concentration as a function of time and inducer concentration. 